In [10]:
!pip install chromadb


  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.11.0
    Uninstalling typing_extensions-4.11.0:
      Successfully uninstalled typing_extensions-4.11.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-objectbox 0.1.0 requires langchain-core<0.2.0,>=0.1.45, but you have langchain-core 0.3.25 which is incompatible.
langchain-openai 0.1.7 requires langchain-core<0.3,>=0.1.46, but you have langchain-core 0.3.25 which is incompatible.
langserve 0.2.3 requires langchain-core<0.3,>=0.1, but you have langchain-core 0.3.25 which is incompatible.


In [1]:
import os

os.environ["GROQ_API_KEY"] = "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"


In [2]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    # GROQ_API_KEY="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",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)



In [3]:
text=input()
respo=llm.invoke(text)
print(respo)

content="It's nice to meet you. Is there something I can help you with or would you like to chat?" additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 36, 'total_tokens': 59, 'completion_time': 0.092, 'prompt_time': 0.009205361, 'queue_time': 1.220972724, 'total_time': 0.101205361}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_b6828be2c9', 'finish_reason': 'stop', 'logprobs': None} id='run-3f7e2791-e47c-4d72-b006-a39d0c4c062b-0' usage_metadata={'input_tokens': 36, 'output_tokens': 23, 'total_tokens': 59}


In [4]:
import chromadb



In [5]:
client = chromadb.Client()

In [6]:
collection=client.create_collection(name="my_collection_1")

In [7]:
collection.add(
    documents=[
        "this document is about new york",
        "this document is about delhi"
    ]
    ,ids=['1','2']
)

In [8]:
all_doc=collection.get()
all_doc

{'ids': ['1', '2'],
 'embeddings': None,
 'documents': ['this document is about new york',
  'this document is about delhi'],
 'uris': None,
 'data': None,
 'metadatas': [None, None],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [9]:
collection.query(query_texts='query is about chhole bhature')

Number of requested results 10 is greater than number of elements in index 2, updating n_results = 2


{'ids': [['2', '1']],
 'embeddings': None,
 'documents': [['this document is about delhi',
   'this document is about new york']],
 'uris': None,
 'data': None,
 'metadatas': [[None, None]],
 'distances': [[1.5588479042053223, 1.8114912509918213]],
 'included': [<IncludeEnum.distances: 'distances'>,
  <IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [10]:
collection.query(query_texts='query is about brooklyn bridge',n_results=2)

{'ids': [['1', '2']],
 'embeddings': None,
 'documents': [['this document is about new york',
   'this document is about delhi']],
 'uris': None,
 'data': None,
 'metadatas': [[None, None]],
 'distances': [[1.095342755317688, 1.571172833442688]],
 'included': [<IncludeEnum.distances: 'distances'>,
  <IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [11]:
from langchain_community.document_loaders import WebBaseLoader

In [12]:
loader=WebBaseLoader("https://careers.ey.com/ey/job/Bengaluru-CA-Qualified-GL-Senior-associate-KA-560016/1090863301/")

In [13]:
page_data=loader.load().pop().page_content

In [14]:
print(page_data)














CA Qualified GL Senior associate Job Details | EY






























Cookie information
Welcome to the EY careers job search site. This website is based on the SuccessFactors software provided by SAP. On this page, functional and optional cookies are used to improve your experience and design our careers site more user-friendly and in line with your needs. In this context, cookies from providers in third countries may also be used and data may be transmitted to providers such as social media services outside the EU. For this we require your consent. By clicking "Accept All Cookies", you agree to these. This also includes your consent to the transmission of certain personal data to third countries, including the USA, in accordance with Art. 49 (1) (a) GDPR. You can select your settings by clicking “Modify Cookie Preferences” to confirm your choices from the optional cookie tracking and selecting the required cookies required to remain on the site. You can change 

In [15]:
from langchain_core.prompts import PromptTemplate

In [16]:
prompt_extract=PromptTemplate.from_template(
    """
    ### scraped text from website:
    {page_data}
    ### instruction:
    the scraped text is from the career's page of website. 
    your job is to extract the job postings and return the in json format containing 
    following keys: 'role','experience','skills',and 'description'.
    only return the valid json.
    ### valid json (no preamble):
    """
)

In [17]:
chain= prompt_extract | llm

In [18]:
res=chain.invoke(input={"page_data":page_data})

In [97]:
print(res.content)

[
  {
    "role": "CA Qualified GL Senior associate",
    "experience": "2-4 years",
    "skills": [
      "Good understanding of Indian Accounting Standards",
      "Working in multinational environment with different time zones",
      "High accuracy and on time delivery",
      "Succinct and structured oral and written communications skills",
      "Expertise in MS Excel, PowerPoint and handling large amount of data sets and tasks",
      "Positive attitude and dependable team player",
      "Strong written and oral communication skills"
    ],
    "description": "The primary responsibility will be overseeing the month-end close, accruals, Audits, intercompany reconciliations, Schedules and related reporting requirement for India entities. You will work closely with other members of the Finance and accounting teams (India and Overseas) and provides accurate and timely information for month-end close. You will also be responsible to liaise and drive the month-end closure by interacti

In [93]:
import time
import random

retries = 5
for attempt in range(retries):
    try:
        res = chain.invoke(input={"page_data": page_data})
        break
    except Exception as e:
        print(f"Attempt {attempt + 1} failed: {e}")
        if attempt < retries - 1:
            time.sleep(2 ** attempt + random.uniform(0, 1))
        else:
            raise


In [99]:
from langchain_core.output_parsers import JsonOutputParser

In [101]:
json_parser=JsonOutputParser()

In [103]:
json_res=json_parser.parse(res.content)

In [105]:
json_res

[{'role': 'CA Qualified GL Senior associate',
  'experience': '2-4 years',
  'skills': ['Good understanding of Indian Accounting Standards',
   'Working in multinational environment with different time zones',
   'High accuracy and on time delivery',
   'Succinct and structured oral and written communications skills',
   'Expertise in MS Excel, PowerPoint and handling large amount of data sets and tasks',
   'Positive attitude and dependable team player',
   'Strong written and oral communication skills'],
  'description': 'The primary responsibility will be overseeing the month-end close, accruals, Audits, intercompany reconciliations, Schedules and related reporting requirement for India entities. You will work closely with other members of the Finance and accounting teams (India and Overseas) and provides accurate and timely information for month-end close. You will also be responsible to liaise and drive the month-end closure by interacting with various stakeholders (India and Over

In [107]:
type(json_res)

list

In [109]:

from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [115]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)


In [121]:
json_res

[{'role': 'CA Qualified GL Senior associate',
  'experience': '2-4 years',
  'skills': ['Good understanding of Indian Accounting Standards',
   'Working in multinational environment with different time zones',
   'High accuracy and on time delivery',
   'Succinct and structured oral and written communications skills',
   'Expertise in MS Excel, PowerPoint and handling large amount of data sets and tasks'],
  'description': 'The primary responsibility will be overseeing the month-end close, accruals, Audits, intercompany reconciliations, Schedules and related reporting requirement for India entities. You will work closely with other members of the Finance and accounting teams (India and Overseas) and provides accurate and timely information for month-end close.'}]

In [19]:
import pandas as pd

In [20]:
df=pd.read_csv(r"C:\Users\ASUS\Downloads\my_portfolio.csv")

In [129]:
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [23]:
print(1+"hi")

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [ ]:
client=chromadb.PersistentClient('vectorstore')
collection=client.get_or_create_collection(name='portfolio')

if not collection.count():
    for _, row in df.iterrows():
        print(_,"----",row)
        collection.add(documents=[row['Techstack']],metadatas=[{'links':row['Links']}],ids=[str(uuid.uuid64())])